***Carlos Figueroa, Assignment 3, Data Management, Spring 2023***

Before starting, it is worth mentioning that only using python limitated the packages we could use for the assignment, so the solutions migth be overcomplicated sometimes, but everything works as intended

**Directory of tables to look on the file:**

movie **(main table left)**

**unique tables with id's:**

actors

languages

countries

locations_temp

genres

**relationship tables:**

movies_and_cast

movies_and_languages

locations

movies_and_genres


In [ ]:
#!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
#!gdown 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi -O /usr/lib/python3.7/lib-dynload/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

gdrivePath="/content/drive/MyDrive/Bhj/"
import sqlite3
import pandas as pd
# create a connection to a database
conn = sqlite3.connect(gdrivePath + "IMDB_horror_movies.db") 

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /usr/lib/python3.7/lib-dynload/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.50M/6.50M [00:00<00:00, 86.2MB/s]


Define formulas that we will use

In [58]:
def execSQL(conn,query):
  conn.execute(query) # execute an SQL query
  conn.commit() # "commit" that query in order to make its action permanent

In [59]:
def allrowsSelect(conn,query):
  cursor = conn.execute(query)
  for row in cursor:
   print(row)

Quick view of the original table

In [60]:
import pandas as pd
pd.read_sql_query('''SELECT * FROM 'IMDB_horror_movies'
limit 2
''',conn)

,title,genres,release_date,release_country,movie_rating,review_rating,movie_runtime,plot,movie_cast,movie_language,filming_locations,budget
0,Gut (2012),Drama| Horror| Thriller,26-Oct-12,USA,None,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...",Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",None
1,The Haunting of Mia Moss (2017),Horror,13-Jan-17,USA,None,NaN,None,"Directed by Jake Zelch. With Nicola Fiore, Bri...",Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,None,"$30,000"


**Drop tables that might exist and create a new table movie, with ID's-------------------------------------------------------------------------------------**

In [61]:
execSQL(conn,"drop table if exists movie")
execSQL(conn,"drop table if exists movie1")

In [62]:
createCMD='''
CREATE TABLE movie (
	movie_id integer PRIMARY KEY autoincrement,
	title text,
  genres object,
	release_date text,
  release_country text,
	movie_rating real,
  review_rating real,
	movie_runtime integer,
  plot text,
  movie_cast text,
  movie_language text,
  filming_locations text,
	budget real
);
'''
execSQL(conn,createCMD)

In [63]:
#populate the query
query='''
INSERT INTO movie (title, genres, release_date, release_country, movie_rating, review_rating, movie_runtime, plot, movie_cast, movie_language, filming_locations, budget)
SELECT title, genres, release_date, release_country, movie_rating, review_rating, movie_runtime, plot, movie_cast, movie_language, filming_locations, budget FROM 'IMDB_horror_movies';
'''
execSQL(conn,query)


In [116]:
query ='''SELECT * 
          FROM movie
'''
pd.read_sql_query(query,conn)
movie = pd.read_sql_query(query,conn)

In [65]:
lst_col = 'genres' 
x = movie.assign(**{lst_col:movie[lst_col].str.split('|')})

In [66]:
data = x.explode('genres')

In [67]:
#load packages to use sql commands to look through the dataset

data_for_console = pysqldf('''select movie_id, genres
from data 
''')

data_for_console.to_sql('products', conn, if_exists='replace', index = False)

**Move the genres to another table and add a genre ID-------------------------------------------------------------------------------------**


In [68]:
execSQL(conn,"drop table if exists genres")
createCMD='''
CREATE TABLE genres (
	genre_id integer PRIMARY KEY autoincrement,
  genre object
);
'''
execSQL(conn,createCMD)

In [69]:
#populate the query
query='''
INSERT INTO genres (genre)
SELECT distinct genres FROM products
'''
execSQL(conn,query)

In [70]:
#this is the table with all the genres inside
query ='''SELECT * 
          FROM genres
'''
pd.read_sql_query(query,conn)

,genre_id,genre
0,1,Drama
1,2,Horror
2,3,Thriller
3,4,Comedy
4,5,Crime
5,6,Action
6,7,Sci-Fi
7,8,Mystery
8,9,Adventure
9,10,Fantasy


In [71]:
#this is the movie query but with  genres separated into rows, matching the same movie_id column 
query ='''SELECT * 
          FROM products
'''
pd.read_sql_query(query,conn)

,movie_id,genres
0,1,Drama
1,1,Horror
2,1,Thriller
3,2,Horror
4,3,Horror
...,...,...
7519,3326,Horror
7520,3327,Action
7521,3327,Adventure
7522,3327,Fantasy


And now that we have a different table for genres, we drop genres from movie

In [72]:
query ='ALTER TABLE movie DROP COLUMN genres;'
execSQL(conn,query)

**Create a table which contains the relationships between the movies and the genres
(using IDs only). A given movie can have more than one genre-------------------------------------------------------------------------------------**


In [73]:
execSQL(conn,"drop table if exists movies_and_genres")

In [74]:
query = ('''create table movies_and_genres as
select movie_id, genre_id from products 
left join genres 
on products.genres = genres.genre''')

execSQL(conn,query)

In [75]:
query ='''SELECT *
          FROM movies_and_genres
'''
pd.read_sql_query(query,conn)

,movie_id,genre_id
0,1,1.0
1,1,2.0
2,1,3.0
3,2,2.0
4,3,2.0
...,...,...
7519,3326,2.0
7520,3327,6.0
7521,3327,9.0
7522,3327,10.0


In [76]:
#lets see if we can indeed retrive genre name with this new table
query ='''SELECT movie_id, genre
          FROM movies_and_genres
          left join genres
          on movies_and_genres.genre_id = genres.genre_id
'''
pd.read_sql_query(query,conn)
#and that is indeed the result we will expect

,movie_id,genre
0,1,Drama
1,1,Horror
2,1,Thriller
3,2,Horror
4,3,Horror
...,...,...
7519,3326,Horror
7520,3327,Action
7521,3327,Adventure
7522,3327,Fantasy


**Now, move the release countries to another table, add a country ID, and replace the
release_country in the table of movies with the country ID as a foreign key**

In [77]:
#create table
execSQL(conn,"drop table if exists countries")
q='''
create table countries(
  country_id integer primary key,
  country text
)
'''
execSQL(conn,q)

In [78]:
#populate
query='''
INSERT INTO countries (country)
SELECT distinct release_country FROM movie;
'''
execSQL(conn,query)

In [79]:
pd.read_sql_query('''SELECT * FROM countries
''',conn)
#so we have created a table with the keys, now we need to set the keys column in movies, substituting countries for keys

,country_id,country
0,1,USA
1,2,Canada
2,3,UK
3,4,Japan
4,5,Spain
...,...,...
67,68,Kazakhstan
68,69,Nigeria
69,70,United Arab Emirates
70,71,Belarus


In [80]:
#so now we add the countries_id column to movie table, matching those keys, with an left outer join
execSQL(conn,"drop table if exists movie1")
query ='''create table movie1 as SELECT *
          FROM movie
          left join countries
          on movie.release_country = countries.country '''
execSQL(conn,query)
#and that is indeed the result we will expect

In [81]:
#and then we delete the original release_country and country which is an exact copy of release_country, leaving only the country keys
query='''
ALTER TABLE movie1
DROP COLUMN release_country
'''
execSQL(conn,query)

In [82]:
query='''
ALTER TABLE movie1
DROP COLUMN country
'''
execSQL(conn,query)

**Drop the plot information from the table**

> Indented block





In [83]:
query='''
ALTER TABLE movie1
DROP COLUMN plot
'''
execSQL(conn,query)

In [84]:
query='''
drop table movie
'''
execSQL(conn,query)

In [85]:
query='''
ALTER TABLE movie1
RENAME TO movie;
'''
execSQL(conn,query)

In [86]:
query ='''SELECT *
          FROM movie
'''
pd.read_sql_query(query,conn)
#and that is indeed the result we will expect

,movie_id,title,release_date,movie_rating,review_rating,movie_runtime,movie_cast,movie_language,filming_locations,budget,country_id
0,1,Gut (2012),26-Oct-12,None,3.9,91 min,Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",None,1
1,2,The Haunting of Mia Moss (2017),13-Jan-17,None,NaN,None,Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,None,"$30,000",1
2,3,Sleepwalking (2017),21-Oct-17,None,NaN,None,Alysia Topol|Anthony Makela|Kelsi Ashley|Patri...,English,"Sudbury, Ontario, Canada",None,2
3,4,Treasure Chest of Horrors II (2013),23-Apr-13,NOT RATED,3.7,82 min,Veronica Ricci|Nicholas Adam Clark|James Culle...,English,"Baltimore, Maryland, USA",None,1
4,5,Infidus (2015),10-Apr-15,None,5.8,80 min,Bonini Mino|Massimo Caratelli|Maurizio Zaffino...,Italian,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...
3322,3323,13 Sins (2014),11-Apr-14,R,6.3,93 min,Mark Webber|Devon Graye|Tom Bower|Rutina Wesle...,English,"New Orleans, Louisiana, USA","$4,000,000",46
3323,3324,Victor Frankenstein (2015),25-Nov-15,PG-13,6.0,110 min,Daniel Radcliffe|Jessica Brown Findlay|Bronson...,English,"London, England, UK","$40,000,000",1
3324,3325,The Exorcism of Molly Hartley (2015),9-Oct-15,UNRATED,4.6,96 min,Sarah Lind|Devon Sawa|Gina Holden|Peter MacNei...,English,"Winnipeg, Manitoba, Canada",None,1
3325,3326,Talon Falls (2017),13-Oct-17,None,4.6,75 min,Brad Bell|Jordyn Rudolph|Ryan Rudolph|Morgan W...,English,None,None,1


**Create a table of movie locations with location ID and add the movie ID as a foreign key
to the movie table**

In [87]:
#create table
execSQL(conn,"drop table if exists locations_temp")
q='''
create table locations_temp(
  location_id integer primary key,
  location text
)
'''
execSQL(conn,q)

In [88]:
#populate
query='''
INSERT INTO locations_temp (location)
SELECT distinct filming_locations FROM movie;
'''
execSQL(conn,query)

In [89]:
#many locations are none, that will count as a NAN for t
query ='''SELECT *
          FROM locations_temp
'''
pd.read_sql_query(query,conn)

,location_id,location
0,1,"New York, USA"
1,2,None
2,3,"Sudbury, Ontario, Canada"
3,4,"Baltimore, Maryland, USA"
4,5,"Tokyo, Japan"
...,...,...
1166,1167,"Plaza Cafe Downtown - 54 Lincoln Ave., Santa F..."
1167,1168,"Eloise Insane Asylum, Westland, Michigan, USA"
1168,1169,"929 South Broadway, Downtown, Los Angeles, Cal..."
1169,1170,"Chatsworth, Los Angeles, California, USA"


In [90]:

execSQL(conn,"drop table if exists location1")
execSQL(conn,"drop table if exists movie1")
query ='''create table location1 as SELECT *
          FROM locations_temp
          left join movie
          on locations_temp.location = movie.filming_locations '''
execSQL(conn,query)
#and that is indeed the result we will expect

In [91]:
execSQL(conn,"drop table if exists locations")

In [92]:
query ='''create table locations as
          SELECT location_id, location, movie_id
          FROM location1
'''
execSQL(conn,query)

In [93]:
query ='''
          SELECT *
          FROM locations
'''
pd.read_sql_query(query,conn)

,location_id,location,movie_id
0,1,"New York, USA",1.0
1,1,"New York, USA",401.0
2,1,"New York, USA",956.0
3,1,"New York, USA",1627.0
4,1,"New York, USA",1997.0
...,...,...,...
2091,1167,"Plaza Cafe Downtown - 54 Lincoln Ave., Santa F...",3317.0
2092,1168,"Eloise Insane Asylum, Westland, Michigan, USA",3319.0
2093,1169,"929 South Broadway, Downtown, Los Angeles, Cal...",3320.0
2094,1170,"Chatsworth, Los Angeles, California, USA",3321.0


Apparently there is a correction, and the real question is Create a table of movie locations with location ID and add the location ID as a foreign key to the movie table. So we will do the corrections

In [ ]:
#since we already have a table with locations and location ID called locations_temp, we will just need to add the location_id to the movie table, as we did for country

In [138]:
#so now we add the countries_id column to movie table, matching those keys, with an left outer join
execSQL(conn,"drop table if exists movie2")
query ='''create table movie2 as SELECT *
          FROM movie
          left join locations_temp
          on movie.filming_locations = locations_temp.location'''
execSQL(conn,query)
#and that is indeed the result we will expect

In [139]:
query ='''
          SELECT *
          FROM movie2
'''
pd.read_sql_query(query,conn)

,movie_id,title,release_date,movie_rating,review_rating,movie_runtime,filming_locations,budget,country_id,location_id,location
0,1,Gut (2012),26-Oct-12,None,3.9,91 min,"New York, USA",None,1,1.0,"New York, USA"
1,2,The Haunting of Mia Moss (2017),13-Jan-17,None,NaN,None,None,"$30,000",1,NaN,None
2,3,Sleepwalking (2017),21-Oct-17,None,NaN,None,"Sudbury, Ontario, Canada",None,2,3.0,"Sudbury, Ontario, Canada"
3,4,Treasure Chest of Horrors II (2013),23-Apr-13,NOT RATED,3.7,82 min,"Baltimore, Maryland, USA",None,1,4.0,"Baltimore, Maryland, USA"
4,5,Infidus (2015),10-Apr-15,None,5.8,80 min,None,None,1,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
3322,3323,13 Sins (2014),11-Apr-14,R,6.3,93 min,"New Orleans, Louisiana, USA","$4,000,000",46,77.0,"New Orleans, Louisiana, USA"
3323,3324,Victor Frankenstein (2015),25-Nov-15,PG-13,6.0,110 min,"London, England, UK","$40,000,000",1,42.0,"London, England, UK"
3324,3325,The Exorcism of Molly Hartley (2015),9-Oct-15,UNRATED,4.6,96 min,"Winnipeg, Manitoba, Canada",None,1,173.0,"Winnipeg, Manitoba, Canada"
3325,3326,Talon Falls (2017),13-Oct-17,None,4.6,75 min,None,None,1,NaN,None


In [140]:
query='''
ALTER TABLE movie2
DROP COLUMN filming_locations
'''
execSQL(conn,query)

In [141]:
query='''
ALTER TABLE movie2
DROP COLUMN location
'''
execSQL(conn,query)

In [142]:
query='''
drop table movie
'''
execSQL(conn,query)

In [143]:
query='''
ALTER TABLE movie2
RENAME TO movie;
'''
execSQL(conn,query)

**Move the languages to another table and add a language ID**


In [94]:
lst_col1 = 'movie_language' 
x1 = movie.assign(**{lst_col1:movie[lst_col1].str.split('|')})

In [95]:
data1 = x1.explode('movie_language')

In [117]:
data_for_console1 = pysqldf('''select movie_id, movie_language
from data1
''')

data_for_console1.to_sql('products1', conn, if_exists='replace', index = False)

In [97]:
#since languague has multiple in one field, we do the same as we did for genre
execSQL(conn,"drop table if exists languages")
q='''
create table languages(
  language_id integer primary key,
  language text
)
'''
execSQL(conn,q)

In [98]:
#populate the query
query='''
INSERT INTO languages (language)
SELECT distinct movie_language FROM products1
'''
execSQL(conn,query)

In [123]:
#this is the table with all the unique languagues
query ='''
          SELECT *
          FROM languages
'''
pd.read_sql_query(query,conn)

,language_id,language
0,1,English
1,2,Italian
2,3,Japanese
3,4,Hindi
4,5,Mandarin
...,...,...
73,74,Saami
74,75,Scottish Gaelic
75,76,Sanskrit
76,77,Aramaic


**Create a table which contains the relationships between the movies and the languages
(using IDs only)**

In [100]:
query ='ALTER TABLE movie DROP COLUMN movie_language;'
execSQL(conn,query)

In [101]:
execSQL(conn,"drop table if exists movies_and_languages")

query = ('''create table movies_and_languages as
select movie_id, language_id from products1
left join languages 
on products1.movie_language = languages.language''')

execSQL(conn,query)

In [102]:
#this is the table with all the unique languagues matching movie_id's
query ='''
          SELECT *
          FROM movies_and_languages
'''
pd.read_sql_query(query,conn)

,movie_id,language_id
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,2.0
...,...,...
3702,3323,1.0
3703,3324,1.0
3704,3325,1.0
3705,3326,1.0


**Remove all unique actors that are in the movie cast to their own table which has
individual actor names and an actor ID. Delete the movie cast column**

In [103]:
#to separate moviecast, we repeat proccess
lst_col2 = 'movie_cast' 
x2 = movie.assign(**{lst_col2:movie[lst_col2].str.split('|')})
data2 = x2.explode('movie_cast')

In [104]:
data_for_console2 = pysqldf('''select movie_id, movie_cast
from data2
''')

data_for_console2.to_sql('products2', conn, if_exists='replace', index = False)

In [114]:
query ='ALTER TABLE movie DROP COLUMN movie_cast;'
execSQL(conn,query)

OperationalError: ignored

In [106]:
#since actors has multiple in one field, we do the same as we did for genre
execSQL(conn,"drop table if exists actors")
q='''
create table actors (
  actor_id integer primary key,
  actor text
)
'''
execSQL(conn,q)

In [107]:
#populate the query
query='''
INSERT INTO actors (actor)
SELECT distinct movie_cast FROM products2
'''
execSQL(conn,query)

In [137]:
#this is the table with all the unique actors
query ='''
          SELECT *
          FROM actors
'''
pd.read_sql_query(query,conn)

,actor_id,actor
0,1,Jason Vail
1,2,Nicholas Wilder
2,3,Sarah Schoofs
3,4,Kirstianna Mueller
4,5,Kaitlyn Mueller
...,...,...
33864,33865,Vili Matula
33865,33866,Boris Bakal
33866,33867,Davorka Tovilo
33867,33868,Dora Lipovcan


**Represent the movie-cast relationship as a table that consists of the movie id and actor
ID.**

In [111]:
execSQL(conn,"drop table if exists movies_and_cast")

query = ('''create table movies_and_cast as
select movie_id, actor_id from products2
left join actors 
on products2.movie_cast = actors.actor''')

execSQL(conn,query)

In [112]:
#this is the table with all the unique actors represented in each movie
query ='''
          SELECT *
          FROM movies_and_cast
'''
pd.read_sql_query(query,conn)

,movie_id,actor_id
0,1,1.0
1,1,2.0
2,1,3.0
3,1,4.0
4,1,5.0
...,...,...
41595,3327,33865.0
41596,3327,33866.0
41597,3327,33867.0
41598,3327,33868.0


**So now we will show the final results**

In [144]:
#for the movie table
# it does not have genre, country, locations, movie_cast and languagues, and it has country id as well as location id as foreign keys
query ='''
          SELECT *
          FROM movie
'''
pd.read_sql_query(query,conn)

,movie_id,title,release_date,movie_rating,review_rating,movie_runtime,budget,country_id,location_id
0,1,Gut (2012),26-Oct-12,None,3.9,91 min,None,1,1.0
1,2,The Haunting of Mia Moss (2017),13-Jan-17,None,NaN,None,"$30,000",1,NaN
2,3,Sleepwalking (2017),21-Oct-17,None,NaN,None,None,2,3.0
3,4,Treasure Chest of Horrors II (2013),23-Apr-13,NOT RATED,3.7,82 min,None,1,4.0
4,5,Infidus (2015),10-Apr-15,None,5.8,80 min,None,1,NaN
...,...,...,...,...,...,...,...,...,...
3322,3323,13 Sins (2014),11-Apr-14,R,6.3,93 min,"$4,000,000",46,77.0
3323,3324,Victor Frankenstein (2015),25-Nov-15,PG-13,6.0,110 min,"$40,000,000",1,42.0
3324,3325,The Exorcism of Molly Hartley (2015),9-Oct-15,UNRATED,4.6,96 min,None,1,173.0
3325,3326,Talon Falls (2017),13-Oct-17,None,4.6,75 min,None,1,NaN


In [118]:
#then we have a table with the unique genre, country, locations, movie_cast and languagues 
query ='''
          SELECT *
          FROM genres
'''
pd.read_sql_query(query,conn)

,genre_id,genre
0,1,Drama
1,2,Horror
2,3,Thriller
3,4,Comedy
4,5,Crime
5,6,Action
6,7,Sci-Fi
7,8,Mystery
8,9,Adventure
9,10,Fantasy


In [131]:
query ='''SELECT *
          FROM locations_temp
'''
pd.read_sql_query(query,conn)

,location_id,location
0,1,"New York, USA"
1,2,None
2,3,"Sudbury, Ontario, Canada"
3,4,"Baltimore, Maryland, USA"
4,5,"Tokyo, Japan"
...,...,...
1166,1167,"Plaza Cafe Downtown - 54 Lincoln Ave., Santa F..."
1167,1168,"Eloise Insane Asylum, Westland, Michigan, USA"
1168,1169,"929 South Broadway, Downtown, Los Angeles, Cal..."
1169,1170,"Chatsworth, Los Angeles, California, USA"


In [120]:
query ='''
          SELECT *
          FROM countries
'''
pd.read_sql_query(query,conn)

,country_id,country
0,1,USA
1,2,Canada
2,3,UK
3,4,Japan
4,5,Spain
...,...,...
67,68,Kazakhstan
68,69,Nigeria
69,70,United Arab Emirates
70,71,Belarus


In [124]:
query ='''
          SELECT *
          FROM languages
'''
pd.read_sql_query(query,conn)

,language_id,language
0,1,English
1,2,Italian
2,3,Japanese
3,4,Hindi
4,5,Mandarin
...,...,...
73,74,Saami
74,75,Scottish Gaelic
75,76,Sanskrit
76,77,Aramaic


In [136]:
#this is the table with all the unique actors
query ='''
          SELECT *
          FROM actors
'''
pd.read_sql_query(query,conn)

,actor_id,actor
0,1,Jason Vail
1,2,Nicholas Wilder
2,3,Sarah Schoofs
3,4,Kirstianna Mueller
4,5,Kaitlyn Mueller
...,...,...
33864,33865,Vili Matula
33865,33866,Boris Bakal
33866,33867,Davorka Tovilo
33867,33868,Dora Lipovcan


And tables representing the relation between movie and genre, locations, languages and cast

In [127]:
query ='''
          SELECT *
          FROM movies_and_genres
'''
pd.read_sql_query(query,conn)

,movie_id,genre_id
0,1,1.0
1,1,2.0
2,1,3.0
3,2,2.0
4,3,2.0
...,...,...
7519,3326,2.0
7520,3327,6.0
7521,3327,9.0
7522,3327,10.0


In [130]:
query ='''
          SELECT *
          FROM locations
'''
pd.read_sql_query(query,conn)

,location_id,location,movie_id
0,1,"New York, USA",1.0
1,1,"New York, USA",401.0
2,1,"New York, USA",956.0
3,1,"New York, USA",1627.0
4,1,"New York, USA",1997.0
...,...,...,...
2091,1167,"Plaza Cafe Downtown - 54 Lincoln Ave., Santa F...",3317.0
2092,1168,"Eloise Insane Asylum, Westland, Michigan, USA",3319.0
2093,1169,"929 South Broadway, Downtown, Los Angeles, Cal...",3320.0
2094,1170,"Chatsworth, Los Angeles, California, USA",3321.0


In [132]:
#this is the table with all the unique languagues matching movie_id's
query ='''
          SELECT *
          FROM movies_and_languages
'''
pd.read_sql_query(query,conn)

,movie_id,language_id
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,2.0
...,...,...
3702,3323,1.0
3703,3324,1.0
3704,3325,1.0
3705,3326,1.0


In [133]:
#this is the table with all the unique actors represented in each movie
query ='''
          SELECT *
          FROM movies_and_cast
'''
pd.read_sql_query(query,conn)

,movie_id,actor_id
0,1,1.0
1,1,2.0
2,1,3.0
3,1,4.0
4,1,5.0
...,...,...
41595,3327,33865.0
41596,3327,33866.0
41597,3327,33867.0
41598,3327,33868.0
